In [2]:
import geopandas as gpd
import os

# Path input dan output
folder_path = r'd:/GeospatialData/01_EMBUNG JABAR/ISeNREM/revisi'
grid_path = os.path.join(folder_path, 'grid30.shp')
pl22_path = os.path.join(folder_path, 'PL22_aoi.shp')
output_path = os.path.join(folder_path, 'olah', 'grid30_pl22_inters.shp')

# Load shapefile
grid_gdf = gpd.read_file(grid_path)
pl22_gdf = gpd.read_file(pl22_path)

# Pastikan sistem koordinat sama
if grid_gdf.crs != pl22_gdf.crs:
    pl22_gdf = pl22_gdf.to_crs(grid_gdf.crs)

# Intersect (spatial overlay) dan filter kolom
intersect_gdf = gpd.overlay(grid_gdf, pl22_gdf, how='intersection')

# Ambil hanya kolom IMGSID dan pl22_rec jika tersedia
columns_to_keep = [col for col in ['IMGSID', 'pl22_rec'] if col in intersect_gdf.columns]
intersect_gdf = intersect_gdf[columns_to_keep + ['geometry']]

# Simpan hasil ke shapefile
intersect_gdf.to_file(output_path)

print(f'Intersect selesai. File disimpan di: {output_path}')

Intersect selesai. File disimpan di: d:/GeospatialData/01_EMBUNG JABAR/ISeNREM/revisi\olah\grid30_pl22_inters.shp


In [3]:
import geopandas as gpd
import os

# Path folder dan file
base_folder = r'd:/GeospatialData/01_EMBUNG JABAR/ISeNREM/revisi'
grid_file = os.path.join(base_folder, 'grid30.shp')
tanah_file = os.path.join(base_folder, 'JENISTANAH_aoi.shp')
output_folder = os.path.join(base_folder, 'olah')
output_file = os.path.join(output_folder, 'grid30_tanah_inters.shp')

# Pastikan folder output ada
os.makedirs(output_folder, exist_ok=True)

# Load shapefile
grid_gdf = gpd.read_file(grid_file)
tanah_gdf = gpd.read_file(tanah_file)

# Samakan CRS jika berbeda
if grid_gdf.crs != tanah_gdf.crs:
    tanah_gdf = tanah_gdf.to_crs(grid_gdf.crs)

# Intersect / overlay
intersect_gdf = gpd.overlay(grid_gdf, tanah_gdf, how='intersection')

# Ambil kolom IMGSID dan JENISTANAH (plus geometry)
columns_to_keep = [col for col in ['IMGSID', 'JENISTANAH'] if col in intersect_gdf.columns]
intersect_gdf = intersect_gdf[columns_to_keep + ['geometry']]

# Simpan ke shapefile
intersect_gdf.to_file(output_file)

print(f'Intersect selesai. Hasil disimpan di:\n{output_file}')

Intersect selesai. Hasil disimpan di:
d:/GeospatialData/01_EMBUNG JABAR/ISeNREM/revisi\olah\grid30_tanah_inters.shp


In [6]:
import os
import geopandas as gpd
from sklearn.preprocessing import MinMaxScaler

# Load shapefile
base_folder = r'd:/GeospatialData/01_EMBUNG JABAR/ISeNREM/revisi'
shp_path = os.path.join(base_folder, 'grid30_par.shp')
gdf = gpd.read_file(shp_path)

# Bersihkan nama kolom: hilangkan spasi & ubah ke huruf kecil
gdf.columns = gdf.columns.str.strip().str.lower()

# Cek nama kolom yang tersedia (untuk memastikan 'ch_mm' dan 'eigen' ada)
print("Kolom tersedia:", gdf.columns.tolist())

# Inisialisasi MinMaxScaler
scaler = MinMaxScaler()

# Normalisasi ch_mm → n_ch
if 'ch_mm' in gdf.columns:
    gdf['n_ch'] = scaler.fit_transform(gdf[['ch_mm']])
else:
    raise KeyError("Kolom 'ch_mm' tidak ditemukan setelah normalisasi nama kolom.")

# Normalisasi eigen → n_eig
if 'eigen' in gdf.columns:
    gdf['n_eig'] = scaler.fit_transform(gdf[['eigen']])
else:
    raise KeyError("Kolom 'eigen' tidak ditemukan setelah normalisasi nama kolom.")

# Simpan shapefile hasil
output_path = os.path.join(base_folder, 'grid30_par_norm.shp')
gdf.to_file(output_path)

print("Normalisasi selesai dan file disimpan sebagai:", output_path)

Kolom tersedia: ['imgsid', 'tanah', 'pl22', 'luas_ha', 'twi', 'ch_mm', 'eigen', 'geometry']
Normalisasi selesai dan file disimpan sebagai: d:/GeospatialData/01_EMBUNG JABAR/ISeNREM/revisi\grid30_par_norm.shp


In [7]:
import os
import geopandas as gpd

# Path folder dan nama file
base_folder = r'd:/GeospatialData/01_EMBUNG JABAR/ISeNREM/revisi'
shp_path = os.path.join(base_folder, 'grid30_par_norm.shp')

# Baca shapefile
gdf = gpd.read_file(shp_path)

# Pastikan kolom yang dibutuhkan tersedia
required_cols = ['n_ch', 'n_eig']
missing = [col for col in required_cols if col not in gdf.columns]
if missing:
    raise KeyError(f"Kolom berikut tidak ditemukan dalam shapefile: {missing}")

# Hitung nilai combidx
gdf['combidx'] = 0.3 * gdf['n_ch'] + 0.7 * gdf['n_eig']

# Simpan ke shapefile baru
output_path = os.path.join(base_folder, 'grid30_par_combidx.shp')
gdf.to_file(output_path)

print("Kolom 'combidx' berhasil ditambahkan dan file disimpan sebagai:", output_path)

Kolom 'combidx' berhasil ditambahkan dan file disimpan sebagai: d:/GeospatialData/01_EMBUNG JABAR/ISeNREM/revisi\grid30_par_combidx.shp


In [1]:
import geopandas as gpd

# Load shapefile hasil sebelumnya
gdf = gpd.read_file("d:/GeospatialData/01_EMBUNG JABAR/ISeNREM/revisi/grid30_par_combidx.shp")

# Fungsi scoring
def score_hujan_eigen(val):
    if val < 0.2: return 1
    elif val < 0.4: return 2
    elif val < 0.6: return 3
    elif val < 0.8: return 4
    else: return 5

def score_twi(val):
    if val < 0: return 1
    elif val <= 5: return 2
    elif val <= 10: return 3
    elif val <= 15: return 4
    else: return 5

# Peta scoring
pl22_scores = {
    'Badan air': 1, 'Hutan': 4, 'Kawasan lindung/konservasi': 4,
    'Lahan terbangun': 2, 'Padang rumput': 3, 'Perkebunan': 4,
    'Pertanian lahan basah': 5, 'Pertanian lahan kering': 3, 'Tambak': 1
}

tanah_scores = {
    'Aluvial': 5, 'Andosol': 3, 'Gleisol': 5, 'Grumusol': 5,
    'Kambisol': 4, 'Litosol': 2, 'Mediteran': 3, 'Podsolik': 3,
    'Regosol': 2, 'Lain-lain': 3
}

# Terapkan fungsi scoring (format integer)
gdf['skor_hujan'] = gdf['combidx'].apply(score_hujan_eigen).astype(int)
gdf['skor_twi'] = gdf['twi'].apply(score_twi).astype(int)
gdf['skor_lahan'] = gdf['pl22'].map(pl22_scores).fillna(3).astype(int)
gdf['skor_tanah'] = gdf['tanah'].map(tanah_scores).fillna(3).astype(int)

# Hitung skor MCDA akhir (float)
gdf['mcda_score'] = (
    0.30 * gdf['skor_hujan'] +
    0.30 * gdf['skor_twi'] +
    0.20 * gdf['skor_lahan'] +
    0.20 * gdf['skor_tanah']
)

# Klasifikasi MCDA
def klasifikasi_mcda(val):
    if val < 2.5: return 'Tidak sesuai'
    elif val < 3.5: return 'Cukup sesuai'
    elif val < 4.5: return 'Sesuai'
    else: return 'Sangat sesuai'

gdf['kelas_mcda'] = gdf['mcda_score'].apply(klasifikasi_mcda)

# Simpan shapefile hasil akhir
output_path = "d:/GeospatialData/01_EMBUNG JABAR/ISeNREM/revisi/grid30_mcda_rev.shp"
gdf.to_file(output_path)

print("✅ Skor MCDA berhasil dihitung dan disimpan sebagai:", output_path)

✅ Skor MCDA berhasil dihitung dan disimpan sebagai: d:/GeospatialData/01_EMBUNG JABAR/ISeNREM/revisi/grid30_mcda_rev.shp
